In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import pytorch_lightning as pl
import torch.nn.functional as F

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),     
    transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=16)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [4]:
size=(32,32)

In [5]:
import torch
import torch.nn as nn

from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import torch.nn.init as init


In [6]:
def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [7]:
device='cuda'

In [8]:
import torch.optim as optim
class ResNet(pl.LightningModule):
    def __init__(self, block, num_blocks, num_classes=10):
        super().__init__()
        self.in_planes = 16
        n_in=self.in_planes
        self.conv1 = nn.Conv2d(3, n_in, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(n_in)
        self.layer1 = self._make_layer(block, n_in, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, n_in*2, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, n_in*4, num_blocks[2], stride=2)

        self.linear = nn.Linear(n_in*4, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred=self(x)
        loss = F.cross_entropy(y_pred, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred=self(x)
        preds = torch.argmax(y_pred,dim=1)
        loss = F.cross_entropy(y_pred, y)
        correct=torch.sum(preds==y)
        total=len(y)
        self.log('val_loss', loss,prog_bar=True)
        return correct,total

    def validation_epoch_end(self, val_step_outputs):
        correct=0
        total=0
        for i,j in val_step_outputs:
            correct+=i
            total+=j
        self.log('acc',correct/total,prog_bar=True)
    def configure_optimizers(self):
            optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
            return {'optimizer':optimizer,
        'lr_sheduler' :optim.lr_scheduler.MultiStepLR(optimizer,[100,150],)
        }


In [9]:
model= ResNet(BasicBlock, [3,3,3])


In [10]:
from torchsummary import summary
summary(model.cuda(), (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 16, 32, 32]           2,304
       BatchNorm2d-4           [-1, 16, 32, 32]              32
            Conv2d-5           [-1, 16, 32, 32]           2,304
       BatchNorm2d-6           [-1, 16, 32, 32]              32
        BasicBlock-7           [-1, 16, 32, 32]               0
            Conv2d-8           [-1, 16, 32, 32]           2,304
       BatchNorm2d-9           [-1, 16, 32, 32]              32
           Conv2d-10           [-1, 16, 32, 32]           2,304
      BatchNorm2d-11           [-1, 16, 32, 32]              32
       BasicBlock-12           [-1, 16, 32, 32]               0
           Conv2d-13           [-1, 16, 32, 32]           2,304
      BatchNorm2d-14           [-1, 16,

In [11]:
trainer = pl.Trainer(check_val_every_n_epoch=1,gpus=1,max_epochs=200,progress_bar_refresh_rate=25)
trainer.fit(model, trainloader,testloader)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type        | Params
---------------------------------------
0 | conv1  | Conv2d      | 432   
1 | bn1    | BatchNorm2d | 32    
2 | layer1 | Sequential  | 14.0 K
3 | layer2 | Sequential  | 51.1 K
4 | layer3 | Sequential  | 203 K 
5 | linear | Linear      | 650   
---------------------------------------
269 K     Trainable params
0         Non-trainable params
269 K     Total params
1.079     Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


In [14]:
from tqdm.notebook import tqdm
import numpy as np
corr=[]
predictions=[]
for x,y in tqdm(testloader):
        y_pred=model(x)
        predictions.append(torch.argmax(y_pred,dim=1).numpy())
        corr.append(y.numpy())

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [15]:
correct_labels=np.hstack(corr)
pred_labels=np.hstack(predictions)

In [16]:
from sklearn.metrics import classification_report
print(classification_report(correct_labels,pred_labels))

              precision    recall  f1-score   support

           0       0.64      0.79      0.71      1000
           1       0.75      0.84      0.79      1000
           2       0.73      0.59      0.66      1000
           3       0.56      0.55      0.55      1000
           4       0.77      0.57      0.65      1000
           5       0.68      0.53      0.59      1000
           6       0.81      0.76      0.79      1000
           7       0.57      0.85      0.68      1000
           8       0.80      0.84      0.82      1000
           9       0.75      0.68      0.72      1000

    accuracy                           0.70     10000
   macro avg       0.71      0.70      0.70     10000
weighted avg       0.71      0.70      0.70     10000



In [18]:
torch.save(model.state_dict(), 'result_model.pt')


Resnet 20 показывает результат выше чем Resnet 18 и Cifar net.
При добавлении Dropout результат улучшается и переобучение исчезает. При изменении параметра "p" в меньшую сторону переобучение становится меньше и точность accuracy возрастает, чем больше значение параметра "p", тем хуже точность accuracy и больше переобучение. L2-регуляризация сглаживает веса, из за чего максимальная точность accuracy немного падает.